In [1]:
import os
%pwd

'd:\\End-to-end-Machine-Learning-Project\\research'

In [2]:
# Change directory to project root
os.chdir("../")

In [3]:
%pwd

'd:\\End-to-end-Machine-Learning-Project'

In [4]:
# setting up enviroment using wes package
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/klan86at/End-to-end-Machine-Learning-Project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="klan86at"
os.environ["MLFLOW_TRACKING_PASSWORD"]="332b51719afa1eb8c7b19f01b95fd0d1def95031"

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    metric_file_name: Path
    all_params: dict
    target_column: str
    mlflow_uri: str


In [6]:
from mlProject.constants import*
from mlProject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__ (
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)


        create_directories([self.config.data_validation.root_dir])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        #schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            metric_file_name=config.metric_file_name,
            all_params=params,
            target_column=params.target_column,
            mlflow_uri="https://dagshub.com/klan86at/End-to-end-Machine-Learning-Project.mlflow",
        )

        return model_evaluation_config
    

In [8]:
import os
import tempfile
import time
from pathlib import Path
from dagshub.upload import Repo
import pandas as pd
import joblib
import mlflow
import mlflow.sklearn
import numpy as np
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from urllib.parse import urlparse
from mlProject.utils.common import save_json

In [ ]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)

        return rmse, mae, r2
    
    def log_into_mlflow(self):
        """Windows-proof MLflow/DagsHub logging"""
        
        # 1. Load data and model
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        # 2. Set tracking URI
        tracking_uri = self.config.mlflow_uri or "file:///mlruns"
        mlflow.set_tracking_uri(tracking_uri.replace('.mlflow', '') + '.mlflow')

        with mlflow.start_run():
            # 3. Log metrics
            X_test = test_data.drop(columns=[self.config.target_column])
            y_test = test_data[self.config.target_column]
            y_pred = model.predict(X_test)
            rmse, mae, r2 = self.eval_metrics(y_test, y_pred)
            mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})

            # 4. Log parameters
            mlflow.log_params(self.config.all_params)

            # 5. Save model temporarily
            temp_dir = tempfile.mkdtemp()
            temp_path = Path(temp_dir) / f"model_{time.time_ns()}.joblib"
            joblib.dump(model, temp_path)

            # 6. Log model with fallback logic
            try:
                mlflow.sklearn.log_model(model, "model")
            except Exception:
                if tracking_uri.startswith("https://"): 
                    from dagshub.upload import Repo
                    repo_name = tracking_uri.split('/')[-1].split('.')[0]
                    try:
                        Repo(
                            os.getenv('MLFLOW_TRACKING_USERNAME'),
                            repo_name,
                            token=os.getenv('MLFLOW_TRACKING_PASSWORD'),
                            branch="main"
                        ).upload(str(temp_path), "models/model.joblib")
                    except Exception as e:
                        print(f"DagsHub upload failed: {e}")
                        mlflow.log_artifact(str(temp_path))
                else:
                    mlflow.log_artifact(str(temp_path))

            # 7. Cleanup temp files with retry
            for _ in range(3):
                try:
                    temp_path.unlink()
                    Path(temp_dir).rmdir()
                    break
                except (PermissionError, FileNotFoundError):
                    time.sleep(0.5)


In [10]:
# Model evaluation pipeline
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2025-07-03 17:27:04,436: INFO: common: YAML file config\config.yaml loaded successfully.]
[2025-07-03 17:27:04,442: INFO: common: YAML file params.yaml loaded successfully.]
[2025-07-03 17:27:04,449: INFO: common: YAML file schema.yaml loaded successfully.]
[2025-07-03 17:27:04,449: INFO: common: Created directory: artifacts/data_validation]
[2025-07-03 17:27:04,457: INFO: common: Created directory: artifacts/model_evaluation]


2025/07/03 17:27:06 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


[2025-07-03 17:27:08,510: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/klan86at/End-to-end-Machine-Learning-Project "HTTP/1.1 200 OK"]
[2025-07-03 17:27:09,414: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/klan86at/End-to-end-Machine-Learning-Project/branches/main "HTTP/1.1 200 OK"]


Uploading files (1) to "klan86at/End-to-end-Machine-Learning-Project"...

[2025-07-03 17:27:09,674: INFO: helpers: Uploading files (1) to "klan86at/End-to-end-Machine-Learning-Project"...]
[2025-07-03 17:27:11,478: INFO: _client: HTTP Request: PUT https://dagshub.com/api/v1/repos/klan86at/End-to-end-Machine-Learning-Project/content/main/ "HTTP/1.1 200 OK"]


Upload finished successfully!

[2025-07-03 17:27:11,494: INFO: helpers: Upload finished successfully!]
🏃 View run grandiose-bee-780 at: https://dagshub.com/klan86at/End-to-end-Machine-Learning-Project.mlflow/#/experiments/0/runs/7a470895b3304783911a693c7c38c39e
🧪 View experiment at: https://dagshub.com/klan86at/End-to-end-Machine-Learning-Project.mlflow/#/experiments/0
